### Testing RAG Applications 📑

## What is RAG (Retrieval-Augmented Generation)?

**RAG** is a method that imprves the Large Language Models (LLMs) by combining them with external knowledge. RAG allows the model to access and use up-to-date, specific information from external data sources.

### When to Use RAG:

You can use RAG for example to add a content of the classes to allow students ask questions, or for ecoomerce to include the info of your products.

- **Private/Proprietary Data**: When you need to query company documents, internal knowledge bases
- **Up-to-date Information**: When the LLM's training data is outdated
- **Domain-Specific Knowledge**: For specialized fields not well-represented in training data
- **Reducing Hallucinations**: By grounding responses in actual documents
- **Cost Efficiency**: Cheaper than fine-tuning for many use cases


### Key Components of RAG:

1. **Document Store**: A collection of documents containing the knowledge you want to query can be a document, code repo
2. **Embeddings**: Vector representations of text chunks that capture semantic meaning
3. **Vector Store**: A database that stores embeddings and enables similarity search
4. **Retriever**: Finds the most relevant documents based on the query
5. **Generator**: The LLM that generates answers using retrieved context

### How RAG Works:

1. The info added to the LLM is break down into chunks and converted to vector
2. The vectors are stored in a database 
3. The user submits a prompt
5. The system tranforms into a numerical format (vector)
5. Use the vector to search into the knowledge base 
6. The info is passed to the language model
7. The answer is returned

## RAG Application Example

In this example the RAG will add some info abotu MCP (Model Context Protocol) and after we can check that the correct answer and context it's returned

1. **Loads data** from the website https://www.descope.com/learn/post/mcp
2. **Chunks the text** into smaller, manageable chunks
3. **Creates embeddings** a embedding is the numerical vector representation of the text that caputre the semantic meaning and stores them in a vector database
4. **Retrieves relevant chunks** when answering questions
5. **Generates answers** using the retrieved context

This approach allows us to answer questions about MCP even though the base LLM might not have been trained on this specific information.

### Setup: 

We will use:

- **LangChain** A framework for developing application powered by languages modesl
- **LangSmith** is LangChain's platform for debugging, testing, and monitoring LLM applications. It provides:
- **OpenAI** The model to add the new info about MCP
- **DeepEval** To test the model
- **Ollama* To use free models from your computer

### API Keys and Environment Variables

Before we start, we need to set up our API keys. This notebook uses:
- **OpenAI API Key**: Required for embeddings and LLM generation
- **LangSmith API Key**: Optional, but recommended for debugging and tracing

You can get a free API key at [https://smith.langchain.com/](https://smith.langchain.com/)

You need to add this API key in a .env file or set as environment variable

In [1]:
import os
from getpass import getpass
import deepeval

# Set up OpenAI API Key (Required)
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

if "CONFIDENT_API_KEY" not in os.environ:
    os.environ["CONFIDENT_API_KEY"] = getpass("Enter your Confident API key: ")

# Set up LangSmith API Key (Optional - suppresses warning)
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = "not-needed"  # Prevents the warning

api_key = os.getenv("CONFIDENT_API_KEY")
deepeval.login_with_confident_api_key(api_key)

🎉🥳 Congratulations! You've successfully logged in! 🙌

Install the next python packages

In [4]:
%pip install -u DeepEval
# Environment and utilities
%pip install python-dotenv  # For loading environment variables

# LangChain packages
%pip install langchain
%pip install langchain-community
%pip install langchain-openai
%pip install langchain-text-splitters
%pip install langchain-chroma  # Already mentioned in the notebook

# Vector stores
%pip install faiss-cpu  # or faiss-gpu if you have GPU support

# Document loading
%pip install beautifulsoup4  # For WebBaseLoader

# OpenAI
%pip install openai
%pip install tiktoken  # For token counting with OpenAI models

# Testing and evaluation
%pip install deepeval

# Additional dependencies that may be required
%pip install numpy
%pip install scikit-learn  # For some vector operations


Usage:   
  /Users/abigailarmijohernandez/GitHub/AI-Testing/venv/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /Users/abigailarmijohernandez/GitHub/AI-Testing/venv/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /Users/abigailarmijohernandez/GitHub/AI-Testing/venv/bin/python -m pip install [options] [-e] <vcs project url> ...
  /Users/abigailarmijohernandez/GitHub/AI-Testing/venv/bin/python -m pip install [options] [-e] <local project path> ...
  /Users/abigailarmijohernandez/GitHub/AI-Testing/venv/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip

### Step 1: Load and Process Documents

The first step in building a RAG application is to load and process your documents. This involves:

1. **Loading documents** from various sources (web, PDFs, databases, etc.)
2. **Splitting text** into smaller chunks for better retrieval
3. **Creating embeddings** (vector representations) of each chunk
4. **Storing embeddings** in a vector database for fast similarity search

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 1. Load data from Web
# WebBaseLoader fetches and parses web pages
loader = WebBaseLoader("https://www.descope.com/learn/post/mcp")
data = loader.load()

# 2. Split text into smaller chunks
# Why? LLMs have context limits, and smaller chunks improve retrieval accuracy
# chunk_size=500: Each chunk will have ~500 characters
# chunk_overlap=0: No overlap between chunks (you might want some overlap in practice)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# 3. Create embeddings and store in vector database
# FAISS: Facebook AI Similarity Search - efficient vector storage and retrieval
# OpenAIEmbeddings: Converts text to vectors using OpenAI's embedding model
vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

print(f"Created vectorstosre with {len(all_splits)} documents")


USER_AGENT environment variable not set, consider setting it to identify your requests.


Created vectorstosre with 39 documents


### Step 2: Create RAG Chain

Now we'll create a RAG chain that combines retrieval with generation. The chain will:
1. Take a user question
2. Retrieve relevant documents from our vector store
3. Format those documents as context
4. Pass both context and question to the LLM
5. Generate an answer based on the retrieved context

#### Using LangChain Hub Prompts

LangChain Hub provides pre-built, tested prompts for common use cases. The `rlm/rag-prompt` is a popular RAG prompt that instructs the model to:
- Answer based on the provided context
- Say "I don't know" if the answer isn't in the context
- Keep answers concise (3 sentences max)

In the next examle we will use a free model llama3.2 

The temperature controls the randomness and creativity of the model's outputs. With lower temperature < .2 will probably returns the same words always. with higher value for example 0.8 will use more random choices and will return results with more variants. 

The max_tokens sets the maximun lenght of the model's repsonse. 

RAG applications generally use low temperature (0 to 0.3) for accuracy 


In [7]:
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.3,
    max_token=500
)

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_llm(
    llm, retriever=vectorstore.as_retriever(), prompt=prompt
)

qa_chain.invoke({"query": "What is Model Context Protocol?"})

{'query': 'What is Model Context Protocol?',
 'result': '<think>\nOkay, the user is asking about the Model Context Protocol (MCP). Let me check the provided context.\n\nFirst, the context mentions that MCP is a protocol by Anthropic, open-source, allowing LLMs to connect with external data and tools. It\'s called a "universal remote" for AI apps. Also, it addresses the NxM problem by standardizing integration, so developers don\'t need custom solutions each time. The key points are standardization, reducing custom integration, and enabling better AI app development.\n\nI need to answer in three sentences max. Start with what MCP is, then its purpose, and finally its impact. Make sure it\'s concise and uses the context provided. Avoid any extra info not in the context. Check for clarity and brevity.\n</think>\n\nThe Model Context Protocol (MCP) is an open-source protocol by Anthropic that enables large language models (LLMs) to connect with external data sources and tools. It addresses 

### RAG that returns the context

LangChain provides multiple ways to create RAG chains. You can use the `create_retrieval_chain` to return the answer and the context (the documents that were added)

1. **Automatically handles retrieval**: The chain manages document retrieval internally
2. **Returns structured output**: Includes both the answer and the retrieved context
3. **Uses a different prompt template**: The `langchain-ai/retrieval-qa-chat` prompt is optimized for conversational interactions

This approach is useful when you want:
- More control over the retrieval process
- Access to both the answer and the source documents
- A more conversational interaction style

In this example I am using the OpenAI that will return better results

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4", 
    temperature=0.3,
    max_tokens=500
)

# See full prompt at https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(vectorstore.as_retriever(), combine_docs_chain)

rag_chain.invoke({"input": "What is Model Context Protocol?"})

{'input': 'What is Model Context Protocol?',
 'context': [Document(id='ae01d6df-14d0-4c4c-b3d0-5eb261626714', metadata={'source': 'https://www.descope.com/learn/post/mcp', 'title': 'What Is the Model Context Protocol (MCP) and How It Works', 'description': 'Learn more about MCP, the open source protocol developed by Anthropic to provide LLMs and AI agents a standardized way to connect with external data sources and tools.', 'language': 'en'}, page_content="What Is the Model Context Protocol (MCP) and How It WorksSkip to main contentArrow RightGet a complimentary copy of the 2025 Gartner Buyers Guide for CIAM. Let's go >Log InUser CircleProductUse CasesDevelopersCustomersResourcesCompanyPricingSign upArrow RightBook a demoArrow RightIdentipediaArrow LeftWhat Is the Model Context Protocol (MCP) and How It Works April 7, 2025Copy linkShare on:Share on LinkedInShare on XShare on BluskyTable of ContentsLLM isolation & the NxM problemOpen table of"),
  Document(id='ae326841-4f3a-4843-a39d-fd

### Using RetrievalQA Chain

Here's yet another way to create a RAG chain using the `RetrievalQA` class. This is a more traditional approach that:

1. **Simplifies chain creation**: Combines retrieval and QA in a single class
2. **Uses the same prompt**: We can reuse the `rlm/rag-prompt` from LangChain Hub
3. **Returns structured output**: The response includes both the query and the result

Key differences:
- Uses `query` as the input key instead of `question` or `input`
- Returns a dictionary with `query` and `result` keys
- Handles the retrieval and formatting internally

This approach is ideal when you want a simple, straightforward RAG implementation without complex chain composition.

### Understanding the Context in RAG Chains

In [10]:
# Define the format_docs function
def format_docs(docs):
    """Format a list of documents into a single string for context."""
    return "\n\n".join(doc.page_content for doc in docs)

# When using hub.pull("rlm/rag-prompt"), the prompt expects:
# - context: The retrieved documents formatted as text
# - question: The user's question

# Let's see how the context is passed through the chain
question = "What is Model Context Protocol?"

# Get the retrieved documents
retrieved_docs = vectorstore.as_retriever().invoke(question)
print(f"Number of retrieved documents: {len(retrieved_docs)}")
print("\nRetrieved documents:")
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:")
    print(f"Content: {doc.page_content[:200]}...")
    print(f"Metadata: {doc.metadata}")

# Format the documents as context
context = format_docs(retrieved_docs)
print(f"\nFormatted context (first 500 chars):\n{context[:500]}...")

# This is what gets passed to the prompt
print(f"\nThe prompt receives:")
print(f"- context: {len(context)} characters of retrieved text")
print(f"- question: {question}")

Number of retrieved documents: 4

Retrieved documents:

Document 1:
Content: What Is the Model Context Protocol (MCP) and How It WorksSkip to main contentArrow RightGet a complimentary copy of the 2025 Gartner Buyers Guide for CIAM. Let's go >Log InUser CircleProductUse CasesD...
Metadata: {'source': 'https://www.descope.com/learn/post/mcp', 'title': 'What Is the Model Context Protocol (MCP) and How It Works', 'description': 'Learn more about MCP, the open source protocol developed by Anthropic to provide LLMs and AI agents a standardized way to connect with external data sources and tools.', 'language': 'en'}

Document 2:
Content: excel at responding to natural language, they’ve been constrained by their isolation from real-world data and systems. The Model Context Protocol (MCP) addresses this challenge by providing a standard...
Metadata: {'source': 'https://www.descope.com/learn/post/mcp', 'title': 'What Is the Model Context Protocol (MCP) and How It Works', 'description': 'Learn m

### Step 3: Testing RAG Application with DeepEval

Now that we have a working RAG application, we need to evaluate its performance. This is crucial because:

1. **RAG Quality Varies**: The quality of answers depends on retrieval accuracy and generation quality
2. **No Ground Truth**: Unlike traditional ML, we often don't have exact "correct" answers
3. **Multiple Dimensions**: We need to evaluate relevance, accuracy, completeness, and more

#### What is DeepEval?

DeepEval is an open-source framework for evaluating LLM applications. It provides:
- Pre-built metrics for common evaluation needs
- Custom metric creation capabilities
- Integration with popular LLM frameworks
- Detailed evaluation reports

#### Creating Test Cases

A test case in DeepEval consists of:
- **Input**: The question asked to the RAG system
- **Actual Output**: What the RAG system returned
- **Expected Output**: What we expect the system to return (optional)
- **Retrieval Context**: The documents retrieved by the RAG system (important for RAG metrics!)


In [11]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

question = "What is Model Context Protocol?"
test_case = LLMTestCase(
  input=question,
  actual_output=qa_chain(question)["result"],
  expected_output="The Model Context Protocol (MCP) addresses this challenge by providing a standardized way for LLMs to connect with external data sources and tools—essentially a “universal remote” for AI apps. Released by Anthropic as an open-source protocol, MCP builds on existing function calling by eliminating the need for custom integration between LLMs and other apps."
)

dataset = EvaluationDataset(test_cases=[test_case])

/var/folders/t0/hls4830123s9kn1cxgy9l_2r0000gn/T/ipykernel_81222/4266105146.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  actual_output=qa_chain(question)["result"],


### Defining Evaluation Metrics

DeepEval provides various metrics to evaluate RAG systems:

#### 1. GEval (General Evaluation)
GEval is a flexible metric that uses LLMs to evaluate outputs based on custom criteria. You can define:
- **Name**: A descriptive name for your metric
- **Criteria**: What the LLM should evaluate
- **Evaluation Parameters**: Which parts of the test case to evaluate

#### 2. Built-in Metrics
- **Answer Relevancy**: Measures if the answer addresses the question
- **Faithfulness**: Checks if the answer is grounded in the retrieved context
- **Contextual Precision**: Evaluates if relevant docs are ranked higher
- **Contextual Relevancy**: Measures how relevant the retrieved context is

In [12]:
from deepeval.test_case import LLMTestCaseParams
from deepeval.metrics import GEval

# Custom metric to evaluate conciseness
# This uses an LLM to judge if the output is concise while complete
concise_metrics = GEval(
    name = "Concise",
    criteria="Assess if the actual output remains concise while preserving all essential information.",
    
    # Only evaluate the actual output (not comparing with expected)
    evaluation_params=[
        LLMTestCaseParams.ACTUAL_OUTPUT
    ]
)

# Custom metric to evaluate completeness
# This checks if all key information is retained in the output
completeness_metrics = GEval(
    name = "Completeness",
    criteria="Assess whether the actual output retains all the key information from the input",
    
    evaluation_params=[
        LLMTestCaseParams.ACTUAL_OUTPUT
    ]
)

If you want to evauate with a local model un comment and execute the next command

In [10]:
#!deepeval set-ollama deepseek-r1:latest

### Evaluation with GEval 

### Running the Evaluation

Now let's run our evaluation with multiple metrics. DeepEval will:
1. Execute each metric on our test case
2. Provide detailed scores and explanations
3. Show which metrics passed or failed based on thresholds

Note: Since our test case doesn't include `retrieval_context`, metrics like Faithfulness and Contextual Precision won't work properly. In a real RAG evaluation, you should always include the retrieved documents.

In [13]:
import deepeval.metrics

deepeval.evaluate(dataset, metrics=[
    completeness_metrics,
    deepeval.metrics.AnswerRelevancyMetric(),
    concise_metrics
] )

✨ You're running DeepEval's latest Completeness (GEval) Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Concise (GEval) Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Completeness (GEval) (score: 0.9932453312614516, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output accurately captures all key points from the input: it identifies MCP as an open-source protocol by Anthropic, explains its purpose of connecting LLMs to external systems, and describes how it solves the NxM problem by standardizing integration. The output is complete, accurate, and retains the necessary level of detail, reflecting the depth of information in the input., error: None)
  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response perfectly addresses the question about Model Context Protocol without any irrelevant information. Great job on staying focused and relevant!, error: None)
  - ✅ Concise (GEval) (score: 0.9377540668798146, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The response effectively captures the essential

✓ Done 🎉! View results on 
]8;id=895440;https://app.confident-ai.com/project/cmayt3ndd0alqpck9015mgsm8/evaluation/test-runs/cmczqo8ka01y1trrq16fjn52j/test-cases\https://app.confident-ai.com/project/cmayt3ndd0alqpck9015mgsm8/evaluation/test-runs/cmczqo8ka01y1trrq16fjn52j/test-]8;;\
]8;id=895440;https://app.confident-ai.com/project/cmayt3ndd0alqpck9015mgsm8/evaluation/test-runs/cmczqo8ka01y1trrq16fjn52j/test-cases\cases]8;;\

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Completeness (GEval)', threshold=0.5, success=True, score=0.9932453312614516, reason='The actual output accurately captures all key points from the input: it identifies MCP as an open-source protocol by Anthropic, explains its purpose of connecting LLMs to external systems, and describes how it solves the NxM problem by standardizing integration. The output is complete, accurate, and retains the necessary level of detail, reflecting the depth of information in the input.', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.0036525000000000004, verbose_logs='Criteria:\nAssess whether the actual output retains all the key information from the input \n \nEvaluation Steps:\n[\n    "Compare the key points in the actual output with the input to identify any missing information.",\n    "Evaluate the completeness of the actual output by verifying if all main ideas

### Creating a Test Dataset

For more comprehensive testing, let's create a dataset with multiple test cases. DeepEval allows you to:

1. **Create Golden datasets**: These are test cases with expected outputs that serve as ground truth
2. **Push datasets to the cloud**: Store and version your test datasets
3. **Pull datasets**: Retrieve datasets for consistent testing across teams

#### Golden Test Cases

Golden test cases are particularly useful for:
- Regression testing
- Comparing different model versions
- Establishing baseline performance

In [14]:
from deepeval.dataset import Golden, EvaluationDataset

test_data = [
    {
        "input": "What is MCP",
        "reference": "The Model Context Protocol (MCP) addresses this challenge by providing a standardized way for LLMs to connect with external data sources and tools—essentially a “universal remote” for AI apps. Released by Anthropic as an open-source protocol, MCP builds on existing function calling by eliminating the need for custom integration between LLMs and other apps."
    },
    {
        "input": "What is Relationship between function calling & Model Context Protocol",
        "reference": "The Model Context Protocol (MCP) builds on top of function calling, a well-established feature that allows large language models (LLMs) to invoke predetermined functions based on user requests. MCP simplifies and standardizes the development process by connecting AI applications to context while leveraging function calling to make API interactions more consistent across different applications and model vendors."
    },
    {
        "input": "What are the core components of MCP, just give the heading",
        "reference":""" 
                    - MCP Client
                    - MCP Servers
                    - Protocol Handshake
                    - Capability Discovery
                """
    }
]

goldens = []
for data in test_data:
    golden = Golden(
        input=data['input'],
        expected_output=data['reference']
    )

    goldens.append(golden)

dataset = EvaluationDataset(goldens=goldens)

dataset.push('test_rag')


### Running Comprehensive Evaluation on Multiple Test Cases

The code below demonstrates how to evaluate multiple test cases with RAG-specific metrics. Notice how the evaluation includes retrieval context, which is essential for metrics like:

- **Faithfulness**: Ensures the answer is grounded in retrieved documents
- **Contextual Precision**: Checks if relevant documents are ranked higher
- **Contextual Relevancy**: Measures the relevance of retrieved documents

These metrics help identify issues like:
- Hallucinations (answers not supported by context)
- Poor retrieval quality
- Irrelevant document retrieval

In [15]:
from deepeval.dataset import Golden, EvaluationDataset
from deepeval.test_case import LLMTestCase
from typing import List
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

def convert_goldens_to_test_cases(goldens: List[Golden]) -> List[LLMTestCase] :
    test_cases = []
    for golden in goldens.goldens:
        response = rag_chain.invoke({"input": golden.input})
        
        test_case = LLMTestCase(
            input=golden.input,
            actual_output=response["answer"],
            expected_output=golden.expected_output,
            retrieval_context=[doc.page_content for doc in response["context"]]
        )
        test_cases.append(test_case)
    return test_cases

dataset = EvaluationDataset()
dataset.pull(alias="test_rag")

data = convert_goldens_to_test_cases(dataset)

concise_metrics = GEval(
    name="Concise",
    model="gpt-4o",  # Specify OpenAI model
    criteria="Assess if the actual output remains concise while preserving all essential information.",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT]
)

deepeval.evaluate(
    data,
    metrics=[
        deepeval.metrics.AnswerRelevancyMetric(),
        deepeval.metrics.FaithfulnessMetric(),
        deepeval.metrics.ContextualPrecisionMetric(),
        deepeval.metrics.ContextualRelevancyMetric(),
        concise_metrics
    ]
)


Output()

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Concise (GEval) Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ❌ Answer Relevancy (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because the output failed to address the input question about what MCP stands for or is, rendering the statements irrelevant to the query., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions, indicating that the actual output is perfectly aligned with the retrieval context. Great job maintaining accuracy and consistency!, error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because all nodes in the retrieval contexts are irrelevant to the input question. The first node discusses the benefits and future developments of MCP, but it does not define what MCP is. The second node highlights the potential of MCP in various applications, yet it fails to explain what

✓ Done 🎉! View results on 
]8;id=276595;https://app.confident-ai.com/project/cmayt3ndd0alqpck9015mgsm8/evaluation/test-runs/cmczqpfe801ybtrrq5oj64pbf/test-cases\https://app.confident-ai.com/project/cmayt3ndd0alqpck9015mgsm8/evaluation/test-runs/cmczqpfe801ybtrrq5oj64pbf/test-]8;;\
]8;id=276595;https://app.confident-ai.com/project/cmayt3ndd0alqpck9015mgsm8/evaluation/test-runs/cmczqpfe801ybtrrq5oj64pbf/test-cases\cases]8;;\

EvaluationResult(test_results=[TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=False, score=0.0, reason='The score is 0.00 because the output failed to address the input question about what MCP stands for or is, rendering the statements irrelevant to the query.', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.00453, verbose_logs='Statements:\n[\n    "The text does not provide specific information on what MCP stands for.",\n    "The text does not provide specific information on what MCP is."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "no",\n        "reason": "The statement does not provide any information about what MCP stands for or is, making it irrelevant to the input question."\n    },\n    {\n        "verdict": "no",\n        "reason": "The statement does not provide any information about what MCP is, making it irrelevant to the input question."\n    }\n]'), MetricData(name='Fai